# Introduction: Simulating Asteroid Impacts with a Poisson Process

In this notebook, we will simulate impacts of Near Earth Asteroids on Earth using a Poisson process. Asteroid impacts can be modeled as a Poisson process because they are independent of one another, the chance of an impact occurring does not change over time, and two events (asteroid impacts) do not occur at the same time. We know the average time between impacts for each size of asteroid (we'll get into this) but not the exact time between impacts and moveover, one impact does not affect the chance of the next. 

We can use a Poisson Process model to calculate the expected number of impacts over an individual's life for each size of asteroid. We can also use the same model to calculate the expected waiting time between asteroid impacts. Although this model - like all models - is an approximation and the data are based on limited observations, we can still get real practice using a Poisson process.

## Poisson Process and Poisson Distribution

The Poisson Process and Poisson Distribution are explained in [this article](https://towardsdatascience.com/the-poisson-distribution-and-poisson-process-explained-4e2cb17d459). Basically, a Poisson process is an appropriate model for events that occur with limited frequency where we know the average frequency (or equivalently the average time between events). The Poisson Distribution allows us to calculate the expected number of events in a time period given the average time between events. The Poisson distribution is a special case of the Binomial Distribution where the number of trials is much greater than the expected number of successes. As asteroid impacts are relatively rare, this assumption fits well. For more on the Poisson Distribution, see [the Wikipedia article](https://en.wikipedia.org/wiki/Poisson_distribution#Basics). 

We'll get into the equations where needed and compare the theoretical values to our values derived from actual data and simulations. 

In [1]:
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 10

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Interactive plotting
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='pearl')

# Used for calculating theoretical value
from scipy.special import factorial

## Data on Asteroid Impacts

Data on Asteroid impact frequency, impact energy, and number of asteroids is from the NASA 2017 Report of the Near Earth Object Science Definition Team. It is available [online here](https://cneos.jpl.nasa.gov/doc/2017_neo_sdt_final_e-version.pdf). The actual data on asteroid impacts is limited, so most of the numbers derived in the report are from simulations. 

#### Main Data Table

The main data we'll use is shown in this table (we have it as a dataframe). To find out the units and meanings of all the columns, go to Table 2-1 in the linked report (on page 26). 

![](images/impact_frequency.PNG)

The main column we are concerned with is the impact frequency $f(n) yr^{-1}$. This is the number of impacts per year for the size of asteroids indicated by $<D>$, the mean diameter in KM. This information is all we will need for a Poisson process! 

The average overall impact frequency for all asteriods is $1.66 x 10^{-9} yr^{-1}$. The rate $f(n) yr^{-1}$ per size category $<D>$ is calculated by multiplying this frequency by the number of objects in the category given by $<N(D_2-D_1)>$. 

According to the overall frequency rate, a single Near Earth Asteroid (NEA) will impact the Earth once in 600 million years. Or, if there are 600 million NEAs, there should be one collison every year. 

__Acronyms__

* NEA: Near Earth Asteroid
* NEO: Near Earth Object
* PHA = Potentially Hazardous Asteroids

With that basic background, let's look at the data.

In [2]:
import warnings; warnings.filterwarnings('ignore', category=FutureWarning)

# Read in the data
df = pd.read_parquet('https://github.com/WillKoehrsen/Data-Analysis/blob/master/poisson/data/asteroid-impact-data-cleaned?raw=true')
df.head()

,range_diameter,diameter,absolute_magnitude,impact_energy,number,impact_frequency,min_diameter,max_diameter
0,.0200–.0251,0.0224,26.0,4.523-01,2850000.0,0.004730,.0200,.0251
1,.0251–.0316,0.0282,25.5,9.02e-01,1350000.0,0.002240,.0251,.0316
2,.0316–.0398,0.0355,25.0,1.80e+00,526000.0,0.000873,.0316,.0398
3,.0398–.0501,0.0447,24.5,3.59e+00,263000.0,0.000437,.0398,.0501
4,.0501–.0631,0.0562,24.0,7.16e+00,116000.0,0.000193,.0501,.0631


I've cleaned things up so we can understand each column. The impact frequency is in impacts per year. To get the average time between impacts, we can take $\frac{1}{\text{impact frequency}}$.

In [3]:
df['time_between_impacts'] = 1 / df['impact_frequency']
df.head()

,range_diameter,diameter,absolute_magnitude,impact_energy,number,impact_frequency,min_diameter,max_diameter,time_between_impacts
0,.0200–.0251,0.0224,26.0,4.523-01,2850000.0,0.004730,.0200,.0251,211.416490
1,.0251–.0316,0.0282,25.5,9.02e-01,1350000.0,0.002240,.0251,.0316,446.428571
2,.0316–.0398,0.0355,25.0,1.80e+00,526000.0,0.000873,.0316,.0398,1145.475372
3,.0398–.0501,0.0447,24.5,3.59e+00,263000.0,0.000437,.0398,.0501,2288.329519
4,.0501–.0631,0.0562,24.0,7.16e+00,116000.0,0.000193,.0501,.0631,5181.347150


# Data Exploration

Before we get to modeling, we'll do a little exploration of the data. This is not strictly necessary, but it's a fun exercise.

### Impact Energy vs Diameter

Let's make a plot of the impact energy by the diameter. The diameters are given in kilometers. Impact energy is in Megatons Equivalent of TNT. For comparison, the Little Boy bomb dropped on Hiroshima had 15 kilotons of TNT equivalent. The largest human bomb ever developed was about 100 Megatons. (See [this Wikipedia article](https://en.wikipedia.org/wiki/TNT_equivalent#Examples))

In [4]:
df.set_index('range_diameter')['impact_energy'].iplot(kind='bar', xTitle='Range of Diameters (KM)', 
                                                      yTitle='Megatons Equivalent TNT', 
                                                      title="Impact Energy vs Diameter")

Here's the same graph with a log scale on the y axis.

In [5]:
df.set_index('range_diameter')['impact_energy'].iplot(kind='bar',
                                                      layout=dict(yaxis=dict(type='log', title='Megatons Equivalent TNT'),
                                                                 xaxis=dict(title='Range of Diameters (KM)'),
                                                      title="Impact Energy vs Diameter Log Scale"))

The relationship between the size of an asteroid and the impact energy is almost exactly linear on a log scale. 

We can see that asteroids have some serious destructive potential. We can also look at some of the other columns, such as the number of asteroids in each size category.

### Number of Asteroid in Each Category

In [6]:
df.set_index('range_diameter')['number'].iplot(kind='bar',
                                                      layout=dict(yaxis=dict(title='Count'),
                                                                 xaxis=dict(title='Range of Diameters (KM)'),
                                                      title="Number of Asteroids vs Diameter"))

Again, a log y axis is probably a better representation of the data.

In [7]:
df.set_index('range_diameter')['number'].iplot(kind='bar',
                                                      layout=dict(yaxis=dict(type='log', title='Count'),
                                                                 xaxis=dict(title='Range of Diameters (KM)'),
                                                      title="Number of Asteroids vs Diameter Log Scale"))

There are only 2 known NEA each in the 2 largest size categories. That's probably a good thing as these have the most desctructive potential! 

The cell below is just for making colorscales in plotly.

In [8]:
import cmocean

# Making colorscales
def cmocean_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append('rgb'+str((C[0], C[1], C[2])))

    return pl_colorscale

colorscales = {cmap_name: cmocean_to_plotly(cmap, len(df)) for cmap_name, cmap in cmocean.cm.cmap_d.items()}

## Interactive Plot for Exploring Data

Here is an interactive plot made with IPython widgets and plotly we can use for exploring the dataset. We are able to choose the variable to plot against the diameter range, whether or not to use a log scale, and the colorscale.

In [9]:
from ipywidgets import interact

@interact
def plot_by_diameter(cscale_name=list(colorscales.keys()), 
                     variable=['impact_frequency', 'impact_energy', 'time_between_impacts',
                               'absolute_magnitude', 'number'],
                    log=True):
    
    colors = colorscales[cscale_name]
    
    # Create the data to plot
    data = [
        go.Scatter(
            x=[row['min_diameter'], row['max_diameter']],
            y=[row[variable], row[variable]],
            name=f'{row["diameter"]:.3f} KM',
            line=dict(color=colors[i]),
        marker=dict(color=colors[i])) for i, row in df.iterrows()
    ]

    variable = variable.replace('_', ' ').title()
    
    # Setup the plot layout
    layout = go.Layout(font=dict(size=16), width=900, height=700,
        xaxis=dict(title='Diameter Range (KM)'),
                      yaxis=dict(title=variable, 
                                 type='log' if log else 'linear'),
                      title=f'Asteroid {variable} vs Range of Diameters')

    # Make the plot and display
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)

interactive(children=(Dropdown(description='cscale_name', options=('thermal', 'thermal_r', 'haline', 'haline_r…

# Asteroid Impact Simulation

Now, let's get to work with the Poisson Process. Here we will run 10000 simluations of a human lifetime (at 100 years) and for each size of asteroid, calculate the expected number of impacts over a lifetime. 

For each size, we have the frequency in asteroids/year, which we can convert into the expected number of impacts by multiplying by the number of years in the period. This gives us $\lambda$, the rate parameter of the Poisson Distribution. $\lambda$ is best described as the expected number of events in the period. 

## Expected Number of Events in a Poisson Process

The expected number of events is given by the following equation.

$$P(n) = \frac{(\lambda)^n e^{-\lambda}}{n!}$$

Where $n$ is the number of events and $\lambda$ is the rate parameter. We can think of the rate parameter as being a product of the frequency and the length of time:

$$\lambda = \frac{events}{time} * {time}$$

In our case, $\frac{events}{time} = \text{impact frequency}$ given in number of impacts per year.

### Simulation in Python

Actually running a Poisson Process simulation is simple in Python with the `np.random.poisson` function which takes an expected value (`lambda`) and a `size` and returns the counts in each process. If our size is 10,000, then it will give us the count of events in each of 10,000 simulations.  

#### Simple Example

To show how the equations work, we'll run a simple example first.

In [10]:
# Simulate 1 lifetime 10,000 times
years = 100
trials = 10_000

# Extract the first frequency and calculate rate parameter
freq = df['impact_frequency'].iloc[0]
lam = freq * years

# Run simulation
impacts = np.random.poisson(lam, size=trials)
impacts

array([0, 0, 2, ..., 0, 0, 4])

The best way to visualize the results is with a histogram showing the number of impacts in each simluation.

In [11]:
pd.DataFrame(impacts)[0].value_counts().iplot(kind='bar', 
                                               xTitle='Impacts', yTitle='Count',
                                              title='Distribution of Impacts')

## All Asteroid Sizes

We can quickly carry out the same procedure for all sizes of asteroids using numpy. In this cell, we get the number of impacts in 10,000 trials of 100 years for all asteroid sizes.

In [12]:
# Each trial is a human lifetime
trials = 10000
years = 100

lambdas = years * df['impact_frequency'].values
impacts = np.random.poisson(lambdas, size=(10000, len(lambdas)))
impacts.shape

(10000, 27)

We expect the average number of impacts for each asteroid size to be close to `lambda` for that asteriod size. The rate parameter is the expected number of events in the interval. 

In [13]:
impacts.mean(axis=0)
lambdas

array([4.682e-01, 2.317e-01, 9.340e-02, 4.460e-02, 1.920e-02, 7.500e-03,
       3.700e-03, 2.300e-03, 1.700e-03, 4.000e-04, 4.000e-04, 7.000e-04,
       5.000e-04, 2.000e-04, 3.000e-04, 2.000e-04, 0.000e+00, 0.000e+00,
       2.000e-04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e-04,
       0.000e+00, 0.000e+00, 0.000e+00])

array([4.73e-01, 2.24e-01, 8.73e-02, 4.37e-02, 1.93e-02, 8.63e-03,
       4.02e-03, 2.16e-03, 1.24e-03, 7.77e-04, 5.61e-04, 4.08e-04,
       2.95e-04, 2.30e-04, 1.68e-04, 1.24e-04, 8.76e-05, 5.11e-05,
       3.87e-05, 2.67e-05, 1.65e-05, 9.10e-06, 6.11e-06, 3.39e-06,
       2.03e-06, 3.37e-07, 3.35e-07])

### Observed and Theoretical Average Values

To make sure we are on the right track, we can plot the observed average values and the theoretical average values for each size of asteroid. We expect the resulting averages to be relatively close.

In [14]:
def plot_mean_data_and_expected(impact_df, lambdas, years, ds=df['range_diameter'], log=False):
    """Plot the expected number of impacts and the observed average number of impacts"""
    
    # Extract the data for average number of impacts
    observed = impact_df.mean(axis=0)
    data = [go.Bar(dict(x=ds, y=observed, name='Observed')),
            go.Bar(dict(x=ds, y=lambdas, name='Theoretical'))]
    
    # Set up the layout
    layout = go.Layout(xaxis=dict(title='Diameter Range (km)'),
                       yaxis=dict(title='Mean Impacts', type='log' if log else 'linear'),
                       title=f'Simulated and Theoretical Average Asteroid Impacts in {years:,} Years')
    
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)

If we take the average value across the 10,000 simulations, we come up with an estimated number of impacts per human lifetime. The estimated number from the simluations should be close to `lambda` for each impact frequency as `lambda` is the expected number of impacts (in other words, the most likely number of impacts).

We can plot both the expected number of impacts and the average number of impacts from the simulations.

In [15]:
impact_df = pd.DataFrame(impacts, columns=df['range_diameter'])
plot_mean_data_and_expected(impact_df, lambdas, years, ds=df['range_diameter'], log=False)

The theoretical and observed values are in close agreement. We can see that the average number of impacts across a human lifetime is less than 1 for all asteroid sizes. 

This might be easier to view with a log scale y axis.

In [16]:
plot_mean_data_and_expected(impact_df, lambdas, years, ds=df['range_diameter'], log=True)

You can expect an average of 0.46 impacts of the smallest asteroid in your lifetime and 0.23 of the next largest. For the largest asteroids, we observed 0 impacts even in 10,000 simulations which agree with the miniscule theoretical chances of around 1 in 1 billion.

Let's look at the maximum number of impacts per lifetime. This isn't exactly the best approach because if we increase the number of simulations, we would expect these numbers to increase. Nonetheless, it's fun to look at the worst-case scenarios.

If you are extremely unlucky, you might see five asteroids impacts of the smallest variey! 

In [17]:
impact_df.max().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Maximum Impacts',
                      title='Maximum Number of Impacts per Human Lifetime')

# Theoretical PMF

To view all the Theoretical PMF, we simply evaluate the Poisson Distribution equation across the asteroid categories for a range of different number of events. What we are left with is the PMF for each size of asteroid.

In [18]:
# Number of impacts
events = np.arange(0.25, impacts.max() + 10, 0.25)

# Calculate the theoretical value for each size of asteroid
theoretical = np.exp(-lambdas) * np.power(lambdas, events.reshape(-1, 1)) / factorial(events).reshape((-1, 1))
theoretical.shape

(59, 27)

To visualize the Theoretical PMF, we'll put them all on the same plot. Each column is a different category of asteroids.

In [19]:
t_df = pd.DataFrame(theoretical, columns=df['diameter'].round(3), 
                    index=events)

t_df.iplot(kind='scatter', mode='lines+markers', xTitle='Number of Events', size=6,
           yTitle='Probability', title='Theoretical PMF for all Asteroid Sizes')

This is pretty uninterseting since the most likely number of asteroids for all cases is 0. 

## Probability Mass Function for Each Asteroid Category

As a useful visualization, we can look at the probability mass function for each size category. This is bascially a historgram of the number of events in each trial. 

For these, we'll show the theoretical curve derived from the Poisson Probability Mass Function (PMF because the number of events is discrete). The theoretical curve is derived from the equation discussed above, 

$$P(n) = \frac{(\lambda)^n e^{-\lambda}}{n!}$$

Where $\lambda$ is the rate parameter derived by multiplying the frequency by the length of the time period.

In [20]:
diameter = '.0316–.0398'
impact_df[diameter].value_counts(normalize=True)

0    0.9101
1    0.0866
2    0.0032
4    0.0001
Name: .0316–.0398, dtype: float64

In [21]:
def plot_pmf_data_and_theoretical(df, impact_df, diameter, years):
    """Plot the observed and theoretical probability mass function (PMF)"""
    
    # Calculate lambda
    freq = float(df.loc[df['range_diameter'] == diameter, 'impact_frequency'])
    lam = freq * years
    
    # Extract the data
    data = 100 * impact_df[diameter].value_counts(normalize=True)
    
    max_events = impact_df[diameter].max() +  3
    
    # Number of events for theoretical distribution
    num_events = np.arange(0, max_events, step=0.25)
    
    # Find the probability according to the Poisson PMF
    prob_num_events = 100 * np.exp(-lam) * np.power(lam, num_events) / factorial(num_events)
    
    # Make the data
    data = [go.Scatter(x=num_events, y=prob_num_events, 
                       mode='lines', name='Theoretical'), 
            go.Bar(dict(x=data.index, y=data.values, name='Observed', 
                              marker=dict(line=dict(width=2)))),
                             ]
    
    # Set up the plot layout
    layout = go.Layout(xaxis=dict(title='Number of Impacts'), 
                       yaxis=dict(title='Probability (%)'),
                       title=f"Observed and Theoretical PMF of Asteroid Impacts {diameter} km in Diameter over {years} Years")
    
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)

In [22]:
plot_pmf_data_and_theoretical(df, impact_df, '.0316–.0398', years=years)

In [23]:
plot_pmf_data_and_theoretical(df, impact_df, '.0200–.0251', years=years)

The simulated data agree nearly exactly with the data. Even for the smallest asteroids, the most likely number of impacts is zero.

## Function for Simulation

Let's make this into a function we can use for any period of years and number of simulations.

In [24]:
def simulate_impacts(df, years, trials=10000):
    """Simulate a Poisson process for asteroid impacts"""
    
    np.random.seed(100)
    
    lambdas = years * df['impact_frequency'].values
    impacts = np.random.poisson(lambdas, size=(trials, len(lambdas)))
    
    # Number of impacts
    events = np.arange(0.25, impacts.max() + 10, 0.25)

    # Calculate the theoretical value for each size of asteroid
    theoretical = np.exp(-lambdas) * np.power(lambdas, events.reshape(-1, 1)) / factorial(events).reshape((-1, 1))
    theoretical_df = pd.DataFrame(theoretical, columns=df['diameter'].round(3), 
                    index=events)
    impact_df = pd.DataFrame(impacts, columns=df['range_diameter'])
    return impact_df, lambdas, theoretical_df

Now we can look at the theoretical number of impacts for all asteroid sizes for more years. This should result in more impacts.

In [25]:
years = 2500

# Generate data
impact_df, lambdas, theo = simulate_impacts(df, years, trials=10000)


# Plot Theoretical PMF of impacts
theo.iloc[:80].iplot(kind='scatter', mode='lines+markers', xTitle='Number of Events', size=6,
           yTitle='Probability', title=f'Theoretical PMF for all Asteroid Sizes for {years} Years')

Now, if we live 2500 years, we can expect significantly more asteroid impacts! 

In [26]:
plot_pmf_data_and_theoretical(df, impact_df, '.0316–.0398', years=years)

In [27]:
plot_pmf_data_and_theoretical(df, impact_df, '.0200–.0251', years=years)

# Asteroid Impacts for all of Human History

Running a simulation for the length of one human life is interesting, but say we are concerned with the well-being of the entire human race. Let's increase the length of time to 2 million years, about as long as the genus Homo (See [this Wikipedia article](https://en.wikipedia.org/wiki/Homo)) has been around and calculate the resulting expected number of impacts. All this requires is increasing the number of years and running the simulation function again.

In [28]:
# Each simulation is the entire history of the genus homo
trials = 10_000
years = 2_000_000

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=RuntimeWarning)
    impact_df, lambdas, theo = simulate_impacts(df, years, trials)

Let's check the mean values against the theoretical (the expected number from lambda).

In [29]:
plot_mean_data_and_expected(impact_df, lambdas, years)

In [30]:
plot_mean_data_and_expected(impact_df, lambdas, years, log=True)

Now let's see if there are any impacts of the largest asteroids in any of the simulations. Again, this is the maximum number of impacts in any simulation which we could artificially increase simply by running more simulations.

In [31]:
impact_df.max().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Maximum Impacts',
                      title='Maximum Impacts per 2 Million Years')

We do see at least one occasion in 10,000 simulations where the largest asteroid hits. This shows how lucky we humans are to be here so show a little appreciation and kindness to your fellow Earthmates! 

### Simulated Data vs Theoretical 

Once again, we can plot the simluations of 2 million years versus the theoretical values.

In [32]:
plot_pmf_data_and_theoretical(df, impact_df, '1.58–2.00', years)

In [33]:
plot_pmf_data_and_theoretical(df, impact_df, '2.51–3.16', years)

### Theoretical Distribution

We can again plot the theoretical Probability Mass Function for each asteroid size. While the most likely number of impacts is still zero for some sizes, we see a noticeable rightward shift in the graph as more impacts become more likely.

In [34]:
# Plot Theoretical PMF of impacts
theo.iloc[:80].iplot(kind='scatter', mode='lines+markers', xTitle='Number of Events', size=6,
           yTitle='Probability', title=f'Theoretical PMF for all Asteroid Sizes for {years} Years')

# Time Between Asteroid Impacts

Next, we'll look at the average number of years between asteroid impacts of different sizes. We'll simulate 100,000,000 (100 million) years and then find the average time between impacts for the different categories.

__Waiting Time Equation__

The waiting time between events in a Poisson process is

$$P(X > time) = e^{-\frac{events}{time} * time}$$

The $\frac{events}{time}$ is the frequency of impacts. The probability of waiting an amount of time decreases exponentially as time increases.

In the cell below, we are simulation each year individually as a Bernoulli variable: either an asteroid hits or it does not. We repeat this 100 million times for each size of asteroid to simluate 100 million years. Then we find the time between asteroid impacts to derive the waiting time.

In [35]:
np.random.seed(100)

# Simulate 100 million years
years = 100_000_000
wait_times = {}

# Simulate each year individually
for freq, diameter in zip(df['impact_frequency'], df['range_diameter']):
    # Each year is a bernoulli trial with probability of success equal to frequency
    a = np.random.choice([0, 1], size=1000000, p=[1-freq, freq])
    
    # Find the time between impacts
    wait_times[diameter] = np.diff(np.where(a == 1)[0])

The wait times currently has the time between successive impacts for each size of asteroid.

In [36]:
wait_times['.0200–.0251']

array([305, 136,  76, ..., 153,  42, 147], dtype=int64)

In [37]:
wait_times['6.31–7.94']

array([], dtype=int64)

There are zero impacts in 100 million years in that category.

## Waiting Time Distribution

Let's plot the distribution of wait times for a single size of asteroids. We'll use a simple histogram and we can compare the simulated values to the theoretical values.

In [38]:
binwidth = 100

def plot_wait_times_observed_and_theoretical(df, wait_times, diameter, binwidth=100, log=False):
    """Plot the expected waiting time observed and theoretical for asteroid impacts"""
    
    freq = float(df.loc[df['range_diameter'] == diameter, 
                        'impact_frequency'])
    
    # Extract the data and put into a dataframe
    wt_df = pd.DataFrame({'wt': wait_times[diameter]})
    
    # Bins for dividing waiting times
    bins = np.arange(0, max(wt_df['wt']), binwidth)
    
    # Create binned wait times
    wt_df['binned_wait_time'] = pd.cut(wt_df['wt'], bins=bins)
    # Count number in each bin and divide by the total
    binned_df = wt_df.groupby('binned_wait_time').count() / len(wt_df)
    
    # Required for plotting
    binned_df.index = binned_df.index.astype(str)

    theoretical_binned_probs = []
    wts = np.arange(0, bins.max() + 1, 1)

    # Theoretical probabilities for each waiting time
    theoretical_probs = np.exp(-freq * wts)
    midpoints = []
    
    # Bin the theoretical waiting times
    for x1, x2 in zip(bins[:-1], bins[1:]):
         theoretical_binned_probs.append(100 * (theoretical_probs[wts == x1] - theoretical_probs[wts == x2])[0])
         midpoints.append((x1 + x2) / 2)
        
    # Create the plot data
    data = [go.Bar(x=binned_df.index if not log else midpoints, y=100 * binned_df['wt'], name='observed'),
            go.Scatter(x=binned_df.index if not log else midpoints, y=theoretical_binned_probs, 
                       mode='markers+lines', 
                       name='Theoretical')]
    # Set up the plot
    layout = go.Layout(xaxis=dict(title='Waiting Time (years)', type='log' if log else 'category'),
                       yaxis=dict(title='Probability (%)'),
                       title=f"Waiting Time between Asteroid Impacts for {diameter} KM Diameter")

    # Show the plot
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)

In [39]:
plot_wait_times_observed_and_theoretical(df, wait_times, '.0200–.0251', binwidth=20, log=False)

In [40]:
plot_wait_times_observed_and_theoretical(df, wait_times, '.0398–.0501', binwidth=500)

The theoretical and observed waiting time frequency distributions match up well. For all asteroid sizes, the mean waiting time is the 1 / frequency of impacts.

In [41]:
plot_wait_times_observed_and_theoretical(df, wait_times, '.0631–.0794', binwidth=5000)

## Mean Waiting Time 

The average weight time should be equal to the frequency of impact. We can plot the weight times and the time between impacts to determine if the observed values match the theoretical.

In [42]:
for key, values in wait_times.items():
    df.loc[df['range_diameter'] == key, 'observed_time_between_impacts'] = values.mean()
    
df.set_index('range_diameter')[['time_between_impacts', 'observed_time_between_impacts']].iplot(kind='bar',
                                                                                               layout=dict(yaxis=dict(type='log',
                                                                                                                     title='Time Between Impacts (Years)'),
                                                                                                          xaxis=dict(title='Diameter Range'),
                                                                                                          title='Average Time between Impacts'))

C:\Users\willk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning:

Mean of empty slice.

C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning:

invalid value encountered in double_scalars



For most of the largest asteroid diameters, there were no waiting times because there were no or only one impact! This shows how rare these events are.

To view the most common impacts, we can use the `Counter` object from the `collections` library.

In [43]:
from collections import Counter
counts = Counter(wait_times['.0251–.0316'])
counts.most_common(10)

[(70, 10),
 (57, 10),
 (34, 9),
 (6, 9),
 (52, 9),
 (175, 9),
 (97, 9),
 (231, 8),
 (124, 8),
 (105, 8)]

In [44]:
counts = Counter(wait_times['.0200–.0251'])
counts.most_common(10)

[(14, 30),
 (17, 29),
 (6, 28),
 (23, 28),
 (22, 26),
 (3, 25),
 (47, 24),
 (34, 24),
 (8, 24),
 (1, 24)]

# Interactive Analysis

For the final part, we can put together a number of the functions in order to create an interactive analysis of asteroid impacts. We'll let the user choose the number of years, the number of simulations and then explore the results.

In [45]:
from ipywidgets import interact_manual

@interact_manual
def interactive_asteroid_impact_analysis(years=(1, 1000, 10), trials=(10, 100_000, 1_000),
                                         diameter=list(df['range_diameter']),
                                         log=False):
    # Run simulation
    impact_df, lambdas, theo = simulate_impacts(df, years, trials)
    # Plot the average values and expected
    plot_mean_data_and_expected(impact_df, lambdas, years, log=log)
    
    # Plot the theoretical nad obserebe PMF for the category of interest
    plot_pmf_data_and_theoretical(df, impact_df, diameter, years)
    # Plot Theoretical PMF of impacts
    theo.iloc[:80].iplot(kind='scatter', mode='lines+markers', xTitle='Number of Events', size=6,
           yTitle='Probability', title=f'Theoretical PMF for all Asteroid Sizes for {years} Years')

interactive(children=(IntSlider(value=491, description='years', max=1000, min=1, step=10), IntSlider(value=490…

# Conclusions

In this notebook, we used a Poisson process to simulate asteroid impacts on Earth. Using a Poisson Process model and the Poisson Distribution, we are able to calculate the expected number of impacts over a time period and the waiting time between impacts. While this may be a somewhat theoretical exercise (because of the limited data), it shows how we can apply a statistical concept to arrive at plausible answers. Furthermore, it gives us a chance to get familiar with running simulations and comparing actual results to expected results from theory. Data science often involves comparing reality with theory and exercises like these can help us learn the basics and how to interpret the outcomes.